In [ ]:
using Revise
import MLJ
import Flux
import CloudMicrophysics.PreprocessAerosolData as PAD
import CloudMicrophysics.ActivationEmulatorModels as AEM

In [ ]:
Standardizer = MLJ.@load Standardizer pkg = MLJModels
EvoTreeRegressor = MLJ.@load EvoTreeRegressor pkg = EvoTrees
NeuralNetworkRegressor = MLJ.@load NeuralNetworkRegressor pkg = MLJFlux

In [ ]:
include("ReadAerosolDataset.jl")

In [ ]:
X_train, Y_train, initial_data = read_aerosol_dataset("datasets/4modal_dataset1_train.csv")
X_train

In [ ]:
model = EvoTreeRegressor()
pipeline =
    PAD.preprocess_aerosol_data_standard |>
    Standardizer() |>
    MLJ.TransformedTargetModel(
        MLJ.TunedModel(
            tuning = MLJ.Grid(goal = 30),
            model = model,
            resampling = MLJ.CV(nfolds = 5),
            range = [
                range(model, :eta, lower = 0.05, upper = 1, scale = :log),
                range(model, :max_depth, lower = 3, upper = 15),
            ],
        ),
        transformer = PAD.target_transform,
        inverse = PAD.inverse_target_transform,
    )
mach = MLJ.machine(pipeline, X_train, Y_train)
MLJ.fit!(mach, verbosity = 2)
MLJ.save("emulators/4modal_evotree_machine_naive.jls", mach)

In [ ]:
pipeline =
    PAD.preprocess_aerosol_data_standard |>
    Standardizer() |>
    MLJ.TransformedTargetModel(
        NeuralNetworkRegressor(
            builder = AEM.NNBuilder([250, 50, 5], [0.3, 0.0, 0.0]),
            optimiser = Flux.Optimise.Adam(0.001, (0.9, 0.999), 1.0e-8),
            epochs = 2000,
            loss = Flux.mse,
            batch_size = 1000,
        ),
        transformer = PAD.target_transform,
        inverse = PAD.inverse_target_transform,
    )
mach = MLJ.machine(pipeline, X_train, Y_train)
MLJ.fit!(mach, verbosity = 2)
MLJ.save("emulators/4modal_nn_machine_naive.jls", mach)

In [ ]:
pipeline =
    PAD.preprocess_aerosol_data_standard |>
    Standardizer() |>
    MLJ.TransformedTargetModel(
        AEM.GPRegressor(5, 2000, false, true, 20),
        transformer = PAD.target_transform,
        inverse = PAD.inverse_target_transform,
    )
mach = MLJ.machine(pipeline, X_train, Y_train)
MLJ.fit!(mach, verbosity = 2)
MLJ.save("emulators/4modal_gp_machine_naive.jls", mach)